# 1. Начало работы с Kafka

Тест подключения к Kafka, проверка возможности отправлять и принимать сообщения

In [2]:
!pip install kafka-python

In [21]:
import json
import time
import random

In [31]:
import hashlib
import uuid

In [22]:
KAFKA_BOOTSTRAP_SERVERS = ['localhost:19092', 'localhost:29092', 'localhost:39092']
TOPIC_NAME = "orders"

In [61]:
from kafka import KafkaProducer, KafkaAdminClient

try:
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        request_timeout_ms=5000,
        retries=3,
        acks='all',
        batch_size=16384,
        linger_ms=10
    )
    print("Подключение успешно!")

    i = 0
    while i <= 30:
        future = producer.send(
            TOPIC_NAME,
            key=str(uuid.uuid4()).encode('utf-8'),
            value=json.dumps({"message": f"order - id {i}", "processed_at":time.time()}).encode("utf-8"),
        )
        
        try:
            record_metadata = future.get(timeout=10)
            print(f"Сообщение {i} отправлено в partition {record_metadata.partition}, offset {record_metadata.offset}")
        except Exception as e:
            print(f"Ошибка отправки сообщения {i}: {e}")

        i += 1
        time.sleep(random.randint(1, 5))
        
    producer.flush()
    producer.close()
except Exception as e:
    print(f"Ошибка: {e}")


Подключение успешно!
Сообщение 0 отправлено в partition 1, offset 0
Сообщение 1 отправлено в partition 1, offset 1
Сообщение 2 отправлено в partition 6, offset 0
Сообщение 3 отправлено в partition 5, offset 0
Сообщение 4 отправлено в partition 4, offset 0
Сообщение 5 отправлено в partition 7, offset 0
Сообщение 6 отправлено в partition 7, offset 1
Сообщение 7 отправлено в partition 6, offset 1
Сообщение 8 отправлено в partition 4, offset 1
Сообщение 9 отправлено в partition 3, offset 0
Сообщение 10 отправлено в partition 5, offset 1
Сообщение 11 отправлено в partition 6, offset 2
Сообщение 12 отправлено в partition 1, offset 2
Сообщение 13 отправлено в partition 6, offset 3
Сообщение 14 отправлено в partition 6, offset 4
Сообщение 15 отправлено в partition 4, offset 2
Сообщение 16 отправлено в partition 3, offset 1
Сообщение 17 отправлено в partition 6, offset 5
Сообщение 18 отправлено в partition 5, offset 2
Сообщение 19 отправлено в partition 7, offset 2
Сообщение 20 отправлено в par

In [37]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    auto_offset_reset="earliest",
    enable_auto_commit=True,
)

print("Успешное подключение к Kafka!")

for message in consumer:
    print(message.value.decode("utf-8"))

consumer.close()

Успешное подключение к Kafka!
{"test": "connection", "timestamp": 1764146458.7503793}
{"message": "order - id 0"}
{"message": "order - id 1"}
{"message": "order - id 2"}
{"message": "order - id 3"}
{"message": "order - id 4"}
{"message": "order - id 5"}
{"message": "order - id 6"}
{"message": "order - id 7"}
{"message": "order - id 8"}
{"message": "order - id 9"}
{"message": "order - id 10"}
{"message": "order - id 11"}
{"message": "order - id 12"}
{"message": "order - id 13"}
{"message": "order - id 0", "processed_at": 1764147135.18417}
{"message": "order - id 1", "processed_at": 1764147140.3124864}
{"message": "order - id 2", "processed_at": 1764147141.334948}
{"message": "order - id 3", "processed_at": 1764147143.3587406}
{"message": "order - id 4", "processed_at": 1764147147.379695}
{"message": "order - id 5", "processed_at": 1764147152.4063165}
{"message": "order - id 6", "processed_at": 1764147157.4295707}
{"message": "order - id 7", "processed_at": 1764147158.4589021}
{"message"

KeyboardInterrupt: 

Почему-то партиция создалась только одна, хотя значение по дефолту было 8. Хорошая возможность попробовать самостоятельно увеличить кол-во партиций. 

Сначала подключимся к админке и посмотрим, что с топиком

In [45]:
from kafka import KafkaAdminClient

KAFKA_BOOTSTRAP_SERVERS = ['localhost:19092', 'localhost:29092', 'localhost:39092']
TOPIC_NAME = "orders"

In [47]:
admin = KafkaAdminClient(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    request_timeout_ms=5000
)
topics = admin.list_topics()
    

In [48]:
print(topics)

['orders']


Есть только один топик, его и нужно проверить

In [55]:
topic_metadata = admin.describe_topics(topics=['orders'])
print(topic_metadata)

[{'error_code': 0, 'topic': 'orders', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 0, 'leader': 1, 'leader_epoch': 0, 'replicas': [1, 2, 3], 'isr': [1, 2, 3], 'offline_replicas': []}], 'authorized_operations': ['READ', 'WRITE', 'CREATE', 'DELETE', 'ALTER', 'DESCRIBE', 'DESCRIBE_CONFIGS', 'ALTER_CONFIGS']}]


Да, действительно одна партиция, значит, можно партиции создать

In [56]:
from kafka.admin import NewPartitions

topic_partitions = {
    'orders': NewPartitions(8)
}
response = admin.create_partitions(topic_partitions)

In [58]:
print(response)

CreatePartitionsResponse_v1(throttle_time_ms=0, topic_errors=[(topic='orders', error_code=0, error_message=None)])


In [59]:
topic_metadata = admin.describe_topics(topics=['orders'])
print(topic_metadata)

[{'error_code': 0, 'topic': 'orders', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 1, 'leader': 2, 'leader_epoch': 0, 'replicas': [2, 3, 1], 'isr': [2, 3, 1], 'offline_replicas': []}, {'error_code': 0, 'partition': 3, 'leader': 1, 'leader_epoch': 0, 'replicas': [1, 2, 3], 'isr': [1, 2, 3], 'offline_replicas': []}, {'error_code': 0, 'partition': 2, 'leader': 3, 'leader_epoch': 0, 'replicas': [3, 1, 2], 'isr': [3, 1, 2], 'offline_replicas': []}, {'error_code': 0, 'partition': 0, 'leader': 1, 'leader_epoch': 0, 'replicas': [1, 2, 3], 'isr': [1, 2, 3], 'offline_replicas': []}, {'error_code': 0, 'partition': 5, 'leader': 1, 'leader_epoch': 0, 'replicas': [1, 2, 3], 'isr': [1, 2, 3], 'offline_replicas': []}, {'error_code': 0, 'partition': 7, 'leader': 3, 'leader_epoch': 0, 'replicas': [3, 2, 1], 'isr': [3, 2, 1], 'offline_replicas': []}, {'error_code': 0, 'partition': 6, 'leader': 2, 'leader_epoch': 0, 'replicas': [2, 3, 1], 'isr': [2, 3, 1], 'offline_replicas': []}, {

[{'error_code': 0, 'topic': 'orders', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 1, 'leader': 2, 'leader_epoch': 0, 'replicas': [2, 3, 1], 'isr': [2, 3, 1], 'offline_replicas': []}, {'error_code': 0, 'partition': 3, 'leader': 1, 'leader_epoch': 0, 'replicas': [1, 2, 3], 'isr': [1, 2, 3], 'offline_replicas': []}, {'error_code': 0, 'partition': 2, 'leader': 3, 'leader_epoch': 0, 'replicas': [3, 1, 2], 'isr': [3, 1, 2], 'offline_replicas': []}, {'error_code': 0, 'partition': 0, 'leader': 1, 'leader_epoch': 0, 'replicas': [1, 2, 3], 'isr': [1, 2, 3], 'offline_replicas': []}, {'error_code': 0, 'partition': 5, 'leader': 1, 'leader_epoch': 0, 'replicas': [1, 2, 3], 'isr': [1, 2, 3], 'offline_replicas': []}, {'error_code': 0, 'partition': 7, 'leader': 3, 'leader_epoch': 0, 'replicas': [3, 2, 1], 'isr': [3, 2, 1], 'offline_replicas': []}, {'error_code': 0, 'partition': 6, 'leader': 2, 'leader_epoch': 0, 'replicas': [2, 3, 1], 'isr': [2, 3, 1], 'offline_replicas': []}, {'error_code': 0, 'partition': 4, 'leader': 3, 'leader_epoch': 0, 'replicas': [3, 1, 2], 'isr': [3, 1, 2], 'offline_replicas': []}], 'authorized_operations': ['READ', 'WRITE', 'CREATE', 'DELETE', 'ALTER', 'DESCRIBE', 'DESCRIBE_CONFIGS', 'ALTER_CONFIGS']}]

In [60]:
for topic_info in topic_metadata:
    if topic_info['error_code'] == 0:  # Успешный запрос
        partition_count = len(topic_info['partitions'])
        print(f"Топик '{topic_info['topic']}' имеет {partition_count} партиций")

🎯 Топик 'orders' имеет 8 партиций


Для запуска консьюмера 

```bash
docker exec -it kafka-1 /opt/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic orders --property "print.key=true"
```